In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import os

In [3]:
import sys
sys.path.append("../")

In [4]:
from huey.utils import get_db_session
db_session = get_db_session()

[2019-07-27 18:45:21,132] {settings.py:182} INFO - settings.configure_orm(): Using pool settings. pool_size=5, pool_recycle=1800, pid=9804
[2019-07-27 18:45:21,280] {base_hook.py:83} INFO - Using connection to: id: huey_dev. Host: localhost, Port: None, Schema: huey_dev, Login: postgres, Password: XXXXXXXX, extra: {}


In [10]:
from app import app, db

In [5]:
from huey.models import Buoy

In [6]:
Buoy()

None - None

In [7]:
db_session.query(Buoy).count()

0

In [8]:
import xml.etree.cElementTree as et
parsed_xml = et.parse("stationmetadata.xml")
station = parsed_xml.getroot()[0]

In [9]:
hist = station.getchildren()[0]

/home/yipstar/anaconda3/envs/drl/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  """Entry point for launching an IPython kernel.


In [10]:
station

<Element 'station' at 0x7f5126e74c78>

In [11]:
hist

<Element 'history' at 0x7f5126e86778>

In [12]:
columns = [m.key for m in Buoy.__table__.columns]

In [13]:
columns

['id',
 'station_id',
 'name',
 'owner',
 'pgm',
 'station_type',
 'lat',
 'lng',
 'point',
 'elevation',
 'hull',
 'anemom_height']

In [14]:
def make_buoy(station):
    buoy = Buoy()
    buoy.station_id = station.get('id')
    buoy.name = station.get('name')
    buoy.owner = station.get('owner')
    buoy.pgm = station.get('pgm')
    buoy.station_type = station.get('type')

    hist = station.getchildren()[0]

    buoy.lat = hist.get('lat')
    buoy.lng = hist.get('lng')
    buoy.point = hist.get('point')
    buoy.elevation = hist.get('elevation')
    buoy.hull = hist.get('hull')
    if hist.get('anemom_height'):
        buoy.anemom_height = hist.get('anemom_height')
    
    point = 'SRID=4326;POINT({} {})'.format(buoy.lat, buoy.lng)
    buoy.point = point
    
    return buoy

In [26]:
buoy = make_buoy(station)

In [15]:
for station in parsed_xml.getroot():
    #print(station)
    buoy = make_buoy(station)
    db_session.add(buoy)

db_session.commit()

/home/yipstar/anaconda3/envs/drl/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  if __name__ == '__main__':


In [16]:
buoy.station_type

'fixed'

In [64]:
point = f"POINT({buoy.lat}, {buoy.lng})"

In [99]:
point = 'SRID=4326;POINT({} {})'.format(buoy.lat, buoy.lng)

In [100]:
buoy.point = point

In [65]:
buoy2 = Buoy(point=point)

In [67]:
buoy2.point

'POINT(44.794, -87.313)'

In [21]:
buoy.point

'SRID=4326;POINT(44.794 -87.313)'

In [102]:
buoy.to_json()

{'id': None,
 'station_id': '0Y2W3',
 'name': 'Sturgeon Bay CG Station, WI',
 'owner': 'U.S.C.G. Marine Reporting Stations',
 'pgm': 'IOOS Partners',
 'station_type': 'fixed',
 'lat': '44.794',
 'lng': '-87.313',
 'point': 'SRID=4326;POINT(44.794 -87.313)',
 'elevation': None,
 'hull': '',
 'anemom_height': '10.0'}

In [90]:
db.session.add(buoy)

In [81]:
Buoy.__table__

Table('buoy', MetaData(bind=None), Column('id', Integer(), table=<buoy>, primary_key=True, nullable=False), Column('station_id', String(length=150), table=<buoy>, nullable=False), Column('name', String(length=564), table=<buoy>), Column('owner', String(length=564), table=<buoy>), Column('pgm', String(length=564), table=<buoy>), Column('station_type', String(length=150), table=<buoy>), Column('lat', Float(), table=<buoy>), Column('lng', Float(), table=<buoy>), Column('point', Geometry(geometry_type='POINT', srid=4326), table=<buoy>), Column('elevation', Float(), table=<buoy>), Column('hull', String(length=100), table=<buoy>), Column('anemom_height', Float(), table=<buoy>), schema=None)

In [1]:
db.session.rollback()

NameError: name 'db' is not defined

In [91]:
db.session.commit()

IntegrityError: (psycopg2.IntegrityError) duplicate key value violates unique constraint "buoy_station_id_key"
DETAIL:  Key (station_id)=(0Y2W3) already exists.

[SQL: INSERT INTO buoy (station_id, name, owner, pgm, station_type, lat, lng, point, elevation, hull, anemom_height) VALUES (%(station_id)s, %(name)s, %(owner)s, %(pgm)s, %(station_type)s, %(lat)s, %(lng)s, ST_GeomFromEWKT(%(point)s), %(elevation)s, %(hull)s, %(anemom_height)s) RETURNING buoy.id]
[parameters: {'station_id': '0Y2W3', 'name': 'Sturgeon Bay CG Station, WI', 'owner': 'U.S.C.G. Marine Reporting Stations', 'pgm': 'IOOS Partners', 'station_type': 'fixed', 'lat': '44.794', 'lng': '-87.313', 'point': None, 'elevation': None, 'hull': '', 'anemom_height': '10.0'}]
(Background on this error at: http://sqlalche.me/e/gkpj)

In [17]:
#db.session.rollback()
db_session.query(Buoy).count()

1424

In [50]:
station.get('id')

'ZBQN7'

In [17]:
buoy = db_session.query(Buoy).filter(Buoy.station_id == station.get('id') ).first()

In [46]:
db.session.query(Buoy).filter(Buoy.station_id == station.get('id') ).count()

0

In [47]:
buoy

In [51]:
buoy = db.session.query(Buoy).first()

In [52]:
db.session.delete(buoy)

In [58]:
db.engine

Engine(postgresql://yipstar@localhost/huey_dev)

In [63]:
Buoy.__table__.columns.keys()

['id',
 'station_id',
 'name',
 'owner',
 'pgm',
 'station_type',
 'lat',
 'lng',
 'point',
 'elevation',
 'hull',
 'anemom_height']

In [19]:
## TODO: fix this, how to get engine from db_session
df = pd.read_sql("select * from buoy", db_session., columns=Buoy.__table__.columns.keys(), index_col="id")

AttributeError: 'Session' object has no attribute 'engine'

In [65]:
df.head()

,station_id,name,owner,pgm,station_type,lat,lng,elevation,hull,anemom_height,point
id,,,,,,,,,,,
4,0Y2W3,"Sturgeon Bay CG Station, WI",U.S.C.G. Marine Reporting Stations,IOOS Partners,fixed,44.794,-87.313,None,,10.0,0101000020E61000001283C0CAA165464079E9263108D4...
5,15319,Sanha - WANE 25947,Chevron,Oil and Gas Industry,oilrig,-5.583,11.833,None,,NaN,0101000020E6100000A245B6F3FD5416C0D122DBF97EAA...
6,18CI3,"Michigan City CG Station, IN",U.S.C.G. Marine Reporting Stations,IOOS Partners,fixed,41.730,-86.910,None,,24.0,0101000020E61000003D0AD7A370DD44400AD7A3703DBA...
7,20CM4,"St. Joseph CG Station, MI",U.S.C.G. Marine Reporting Stations,IOOS Partners,fixed,42.090,-86.490,None,,24.0,0101000020E6100000EC51B81E850B45408FC2F5285C9F...
8,21346,"21346 380km East of Iwate, Japan",Japanese Meteorological Agency,Tsunami,dart,40.303,146.192,None,,NaN,0101000020E6100000105839B4C8264440A01A2FDD2446...


In [70]:
df['station_type'].unique()

array(['fixed', 'oilrig', 'dart', 'other', 'moored buoy', 'buoy'],
      dtype=object)

In [71]:
len(df[df['station_type'] == "buoy"])

55